In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
import kagglehub
path = kagglehub.dataset_download("quora/question-pairs-dataset")

c:\Users\saumy\OneDrive\Desktop\PROJECT\Python\NLP\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv(f"{path}/questions.csv")

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [7]:
new_df = df.sample(30000)

In [8]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [10]:
ques_df = new_df[['question1', 'question2']]
ques_df.head()

,question1,question2
132947,What is the cost of a commercial wifi deployme...,Why does my wifi switch from 2.4ghz to 5ghz?
102098,What are antonyms? What are examples of antonyms?,What are examples of antonyms with sentence ex...
326592,What is EssayTyper?,How can Donald Trump improve Obamacare?
276768,Where is the Royal Enfield Service Center in N...,Where is the Royal Enfield Service Center in I...
2295,Is there any law that prohibits a naval office...,Are there any laws that prohibit a sports leag...


In [11]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=3000)
questions = list(ques_df['question1']) + list(ques_df['question2'])
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(), 2)

In [13]:
temp_df1 = pd.DataFrame(q1_arr, index=ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index=ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)

In [14]:
temp_df.shape

(30000, 6000)

In [15]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
132947,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102098,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
326592,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
276768,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2295,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [17]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
132947,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102098,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
326592,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
276768,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2295,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
from sklearn.model_selection import train_test_split

x = temp_df.drop(columns=['is_duplicate'])
y = temp_df['is_duplicate']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()
rf.fit(x_train,y_train)

y_pred = rf.predict(x_test)
accuracy_score(y_test, y_pred)

0.744